# Projet Data Mining - Système de recommandation d'images
### Auteur: Rodolphe Lajugie | Valentin Barriquand
#### Date: 07/02/2023


In [4]:
# Projet Data Mining
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

# Importation des librairies
import traceback
import sys
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.request
import PIL.Image as Image
import numpy
import math
import matplotlib.pyplot as plot
#import KMeans
import time
import json
from cluster import MiniBatchKMeans
from webcolors import rgb_to_name
import pandas as pd

# Variables globales
i = 0
url = "https://query.wikidata.org/sparql"

# Ajouter en plus la selections des type d'objet pour les ajoouter au metadata
queries = ["""select  ?image {
  ?voiture wdt:P31 wd:Q1420;
           wdt:P18 ?image
  }
limit 100""",
"""select  ?image {
  ?gville wdt:P31 wd:Q1549591;
           wdt:P18 ?image
  }
limit 100""",


"""select  ?image {
  ?jeu wdt:P31 wd:Q7889;
           wdt:P18 ?image
  }
limit 100""",

"""select  ?image {
  ?landscape wdt:P31 wd:Q107425;
           wdt:P18 ?image
  }
limit 100""",

"""select  ?image{
  ?dogs wdt:P31 wd:Q144;
           wdt:P18 ?image
  }
limit 100"""]

querie2 = """select  ?image {
  ?people wdt:P31 wd:Q5;
           wdt:P18 ?image
  }

limit 50"""

ImportError: cannot import name 'MiniBatchKMeans' from 'cluster' (/opt/homebrew/lib/python3.11/site-packages/cluster/__init__.py)

## Récupération des résultats

In [ ]:
def get_results(url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [ ]:
def get_res():
    res = []
    for q in queries : 
        gettedQ = get_results(url, q)
        res.extend(gettedQ["results"]["bindings"])
    return res

## Téléchargement d'images
Et sauvegarde dans le dossier images sous le nom imageX.jpg

In [ ]:
def download_image(url,index):
    name = "image" + str(index) + ".jpg"
    urllib.request.urlretrieve(url, "images/" + name)

## Suppression des images

In [ ]:
def delete_donnees():
    if len(os.listdir("images")) != 0 or len(os.listdir("metadonnees")) != 0:
        for file in os.listdir("images"):
            os.remove(os.path.join("images", file))
        for file in os.listdir("images2"):
            os.remove(os.path.join("images2", file))
        for file in os.listdir("metadonnees"):
            os.remove(os.path.join("metadonnees", file))
    else:
        print("Les dossiers sont vide")

## Redimensionnement des images

In [ ]:
def resizeIMG(img):
    factor = 1
    if img.width > 1028 or img.height > 1028:
        if img.height > img.width:
            factor = 1028 / img.height
        else:
            factor = 1028 / img.width
    return img.resize((int(img.width * factor), int(img.height * factor)))

## Calcul la distance entre deux couleurs

In [ ]:
def get_distance(color1, color2):
    return math.sqrt((color1[0]-color2[0])**2 + (color1[1]-color2[1])**2 + (color1[2]-color2[2])**2)

## Récupération du format des images
Portrait, paysage ou carré

In [ ]:
def getformat(image):
    # Permets de récupérer le format d'une image
    imgfile = Image.open("images/"+image)
    long = imgfile.size[0]
    larg = imgfile.size[1]
    if long > larg:
        format = "paysage"
    elif larg > long:
        format = "portrait"
    else:  
        format = "carre"
    return format

## Récupération des couleurs dominantes
Pour cela on utilise la fonction MiniBatchKmeans de la librairie sklearn.cluster

In [ ]:
def getcolors(image):
    n_clusters = 2
    imgfile = Image.open("images/"+image)
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = MiniBatchKMeans(n_clusters,n_init = 'auto')
    clusters.fit(numarray)
    npbins = numpy.arange(0, n_clusters + 1)
    histogram = numpy.histogram(clusters.labels_, bins=npbins)
    labels = numpy.unique(clusters.labels_)

    hist_decres = sorted(histogram[0], reverse=True)

    colors = []

    for i in range(n_clusters):
        colors.append([int(clusters.cluster_centers_[i][0]),int(clusters.cluster_centers_[i][1]),int(clusters.cluster_centers_[i][2])])
        
    
    return colors

## Récupération des métadonnées
Ces métadonnées sont ensuite sauvegardées dans un fichier json.

In [ ]:
def meta(nom,ind):
    global data
    data = {"images": []}
    if not os.path.exists("metadonnees"):
        os.makedirs("metadonnees")

    for i in range(len(os.listdir(nom))):
        try:
            imgfile = Image.open(nom + "/image"+str(i)+".jpg")
            imgscale = resizeIMG(imgfile)
            if len(imgscale.getbands()) < 3:
                continue
            img = {
                "nom": f"image{i}.jpg",
                "mode": imgscale.mode,
                "size": imgscale.size,
                "extension": imgfile.format,
                "format": getformat(f"image{i}.jpg"),
                "colors" : getcolors(f"image{i}.jpg"),
            }
            imgfile.close()
            data["images"].append(img)
        except Exception as e:
            traceback.print_exc()
            print(f"image{i}.jpg", "Image non valide:", e)

    with open("metadonnees/data"+str(ind)+".json", "w") as monfichier:
        json.dump(data, monfichier, indent=4)

## Fonction de vérification
Cette fonction permet de vérifier si le dossier images existe. Si ce n'est pas le cas, elle le crée., Si le dossier existe, on vérifie qu'il est vide. s'il n'est pas vvide, on affiche un message d'erreur. Et s'il est vide, on télécharge les images.

In [ ]:
def full_path():
    global i
    if not os.path.exists("images"):
        os.makedirs("images")
    if len(os.listdir("images")) == 0:
        for result in get_res():
            download_image(result["image"]["value"],i)
            time.sleep(0.01)
            i+=1
        meta("images",0)
    else:
        print("Le dossier n'est pas vide")

## Conversion de couleurs du format RGB au format hexadécimal

In [ ]:
def rgb_to_hex(r,g,b):
    # Convertit une couleur RGB en hexadécimal
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

## Suppression des images
Si l'image n'est pas exploitable, on la supprime.

In [ ]:
def removeBigfile(image):
    # Permet de supprimer une image
    try:
        if os.path.exists("images/"+image):
            os.remove("images/"+image)
        else : 
            print("Image non existante")
    except:
        print("Erreur lors de la suppression de l'image")

## Main

In [ ]:
def main():
    # Cette fonction permet de lancer le programme
    usr = int(input("Que voulez-vous faire ? \n 1 - La totale \n 2 - Créer les métadonnées \n 3 - Supprimer les données \n 4 - Quitter \n"))
    if usr == 1:
        full_path()
    elif usr == 3:
        delete_donnees()
    elif usr == 2:
        meta("images",0)
    elif usr == 4:
        print("Au revoir")
        exit()
    else:
        print("Veuillez entrer un nombre entre 1 et 4")



In [ ]:
main()

# Classe utilisateur 

Cette classe sera la pour stocker les informations de l'utilisateur. Elle contiendra les informations suivantes:
* L'orientation préférée de l'utilisateur
* La couleur préférée de l'utilisateur parmis la liste de webcolors



In [ ]:
import string
from random import randint


class user():
    def __init__(self): 
        self.favcolor = [randint(0,255),randint(0,255),randint(0,255)]
        self.favformat = self.setfavformat()
        self.favtheme = None


    def setfavformat(self):
        temp = int(input("Choisissez un format (1 : paysage, 2 : portrait ou 3 : carré) : "))
        if temp == 1:
            favformat = str('paysage')
        elif temp == 2:
            favformat = str('portrait')
        elif temp == 3:
            favformat = str('carre')
        else:
            print("Veuillez entrer un nombre entre 1 et 3")
            self.setfavformat()
        return favformat


    def getfavformat(self):
        return self.favformat



# Classe main
Cette classe permet de lancer le programme. Elle contiendra les informations suivantes:
* L'utilisateur
* Les données des images (orientation, couleur, nom, chemin)

In [ ]:
class main():
    def __init__(self):
        self.usr = user()
        self.data = self.getdata()

    def getdata(self):
        with open("metadonnees/data0.json") as datafile:
            data = json.load(datafile)
        return data
    
    def showimg(self):
        for i in range(len(self.data["images"])):
            if self.data["images"][i]["format"] == self.usr.getfavformat() and get_distance(self.usr.favcolor,self.data["images"][i]["colors"][0])<150 :
                img = Image.open("images/"+self.data["images"][i]["nom"])
                img.show()
                img.close()
            else:
                print("Image non correspondante à vos critères")
                

x = main()
x.showimg()

: 

# Code permettant de génerer l'arbe de décision

In [ ]:


#creating dataframes
import graphviz
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
import pydotplus
from IPython.display import Image, display

result = ['Favorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite',
        'NotFavorite']

dataframe = pd.DataFrame(x.data["images"], columns=['colors', 'format', 'size'])

resultframe = pd.DataFrame(result, columns=['favorite'])
dataframe[['colors1','colors2']] = pd.DataFrame(dataframe["colors"].tolist(), index= dataframe.index)
dataframe[['c1R','c1G','c1B']] = pd.DataFrame(dataframe["colors1"].tolist(), index= dataframe.index)
dataframe[['c2R','c2G','c2B']] = pd.DataFrame(dataframe["colors2"].tolist(), index= dataframe.index)
dataframe[['width','height']] = pd.DataFrame(dataframe["size"].tolist(), index= dataframe.index)
dataframe.drop(columns=['colors','size','colors1','colors2'], inplace=True)
print(dataframe)
le2 = LabelEncoder()
dataframe['format'] = le2.fit_transform(dataframe['format'])

le5 = LabelEncoder()
resultframe['favorite'] = le5.fit_transform(resultframe['favorite'])

#Utilisation d'arbre de décisision
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe[0:len(dataframe)])

dot_data = tree.export_graphviz(dtc, out_file=None,
        feature_names=dataframe.columns,
        filled=True, rounded=True, 
        class_names = 
        le5.inverse_transform(
        resultframe.favorite.unique())
        ) 
graph = graphviz.Source(dot_data) 

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
img = Image(pydot_graph.create_png())
display(img)

# Recherche des deux couleurs préférées de l'utilisateur d'après les données de l'arbre de décision
* On fait simplement la moyenne des deux couleurs les plus présentes dans les images préférées de l'utilisateur

In [ ]:
couleur1 = [0,0,0]
couleur2 = [0,0,0]
nb = 0
for i in range(len(dataframe)):
    if resultframe['favorite'][i] == 0:
        nb +=1
        couleur1[0] += dataframe['c1R'][i]
        couleur1[1] += dataframe['c1G'][i]
        couleur1[2] += dataframe['c1B'][i]
        
        couleur2[0] += dataframe['c2R'][i]
        couleur2[1] += dataframe['c2G'][i]
        couleur2[2] += dataframe['c2B'][i]
    else :
        None

couleur1[0] = round(couleur1[0]/nb,0)
couleur1[1] = round(couleur1[1]/nb,0)
couleur1[2] = round(couleur1[2]/nb,0)

couleur2[0] = round(couleur2[0]/nb,0)
couleur2[1] = round(couleur2[1]/nb,0)
couleur2[2] = round(couleur2[2]/nb,0)

couleur = [couleur1,couleur2]
print("Les deux couleurs préférées de l'utilisateur sont: ",couleur)


def afficher_couleur(RGB):
    couleur = [v/255 for v in RGB] # Convertir les valeurs de 0-255 à 0-1
    plot.figure(figsize=(2,2))
    plot.axes().set_aspect('equal', 'datalim')
    plot.axes().add_patch(plot.Rectangle((0, 0), 1, 1, facecolor=couleur, edgecolor='none'))
    plot.axis('off')
    plot.show()

for lst in couleur:
    afficher_couleur(lst)


In [ ]:
print(dataframe)

On va ensuite télecharger de nouvelles images pour en sortir celle que l'utilisateur est succeptible d'aimer.

In [ ]:
import PIL.Image as Image
def get_res2():
    res = []
    gettedQ = get_results(url, querie2)
    res.extend(gettedQ["results"]["bindings"])
    return res


def download_image2(url,index):
# Cette fonction permet de télécharger une image à partir d'une url
# et de la sauvegarder dans un dossier images sous le nom imageX.jpg
    name = "image" + str(index) + ".jpg"
    urllib.request.urlretrieve(url, "images2/" + name)


def full_path2():
# Cette fonction permet de vérifier si le dossier images existe
# si le dossier n'existe pas, il est créé
# si le dossier existe, on vérifie qu'il est vide
# si le dossier n'est pas vide, on affiche un message
# si il est vide, on télécharge les images
    var = 0
    if not os.path.exists("images2"):
        os.makedirs("images2")
    if len(os.listdir("images2")) == 0:
        for result in get_res2():
            download_image2(result["image"]["value"],var)
            time.sleep(0.01)
            var+=1
        try :
            meta("images2",1)
        except:
            None
    else:
        print("Le dossier n'est pas vide")

full_path2()

    

In [ ]:
with open("metadonnees/data1.json") as datafile:
    data = json.load(datafile)


for i in range(len(data["images"])):
    if (get_distance(data["images"][i]["colors"][0],couleur[0])<50 and get_distance(data["images"][i]["colors"][1],couleur[1])<50):
        try:
            img = Image.open("images2/"+data["images"][i]["nom"])
            img.show()
            img.close()
        except:
            None

Fonctions rédigées mais non utilisées

In [ ]:
# def similar_color(color):
#     # Fonction qui permet de récupérer la couleur la plus proche de la couleur donnée
#     list_color = get_list_colors()
#     min_distance = float('inf')
#     for color_name, *rgb in list_color:
#         distance = get_distance(color, rgb)
#         if distance < min_distance:
#             min_distance = distance
#             closest_color = color_name
#     return closest_color
  


# def get_list_colors():
# # Fonction qui permet de récupérer la liste des couleurs connu de la librairie webcolors
#     list_color = []
#     for r in range(256):
#         for g in range(256):
#             for b in range(256):
#                 try:
#                     lst = [rgb_to_name((r,g,b), spec='css3'),r,g,b]
#                     list_color.append(lst)
#                 except ValueError:
#                     pass
#     return list_color

"""
def similar_color(color):
    # Fonction qui permet de récupérer la couleur la plus proche de la couleur donnée
    list_color = get_list_colors()
    min_distance = float('inf')
    for color_name, *rgb in list_color:
        distance = get_distance(color, rgb)
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name
    return closest_color

def get_list_colors():
# Fonction qui permet de récupérer la liste des couleurs connu de la librairie webcolors
    list_color = []
    for r in range(256):
        for g in range(256):
            for b in range(256):
                try:
                    lst = [rgb_to_name((r,g,b), spec='css3'),r,g,b]
                    list_color.append(lst)
                except ValueError:
                    pass
    return list_color
    
def similar_color(color):
        # Fonction qui permet de récupérer la couleur la plus proche de la couleur donnée
    min_distance = float('inf')
    for color_name, rgb in colors.items():
        distance = get_distance(color, rgb)
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name
    return closest_color

"""